In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import julia 
import time
import random
import struct
import pyreadstat
import julian

from tletools import TLE
from astropy.time import Time

from astropy.coordinates import CartesianDifferential,CartesianRepresentation,ITRS,GCRS,TEME,EarthLocation
from astropy import units as u
from mpl_toolkits.basemap import Basemap
from scipy.io import readsav
from scipy.interpolate import RegularGridInterpolator as RGI,interp1d,interp2d,splrep,splev
from spacepy import pycdf
from scipy.integrate import cumtrapz,simps
from scipy.stats import pearsonr
from scipy.signal import resample,correlate
from numpy import floor,pi,corrcoef

%matplotlib widget
os.environ["CDF_LIB"] = "C:\CDF_Distribution\cdf38_0-dist\lib"
m_p = 1.6726219 * (10**-27)
m_o = 16*m_p


In [2]:
%load_ext julia.magic
%julia using Dates;
%julia using LinearAlgebra;
%julia using DataFrames;
# %julia cd("C://Users//soumy//OneDrive - The University of Texas at Dallas//Documents//MURI Project//SatelliteToolbox.jl-master//src//")
# %julia include("SatelliteToolbox.jl");
%julia using SatelliteToolbox

Initializing Julia interpreter. This may take some time...


In [3]:
# filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI Project\\Data\\Temp\\"
# os.chdir(filepath)
# filename = "GITM_epoch_2_min_june_2015.csv"
# gitm_epoch = pd.read_csv(filename)
# gitm_epoch.index = pd.DatetimeIndex(gitm_epoch["Unnamed: 0"])
# del gitm_epoch["Unnamed: 0"]
# gitm_epochs = gitm_epoch.index

In [3]:
filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\"
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI Project/Data/Temp/'
os.chdir(filepath)

filename = 'CNOFS_june_2015.csv'
Data = pd.read_csv(filename)
Data.index = pd.DatetimeIndex(Data['Unnamed: 0'])
CNOFS_Data = Data.drop('Unnamed: 0',axis=1)

In [4]:
sat = 'CNOFS'
year = 2015;month = 6;
start = CNOFS_Data.index[0]; 
stop = CNOFS_Data.index[-1];


In [5]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\TLE\\'
os.chdir(filepath)
filename = 'CNOFS.txt'
f = open(filename,'r')
Data = f.readlines()
f.close()

In [6]:

epochs = []
for i in range(int(len(Data)/2)):
    tle_str = 'CNOFS\n' + Data[2*i] + Data[2*i + 1]
    tle_str = tle_str.strip().splitlines()
    tle = TLE.from_lines(*tle_str)
    epochs.append(dt.datetime(tle.epoch_year,1,1,0,0,0) + dt.timedelta(days = tle.epoch_day - 1))

epochs = pd.DatetimeIndex(epochs)
idx0 = np.where(epochs <= start)[0][-1]
idx1 = np.where(epochs >= stop)[0][0]
idx = list(range(idx0,idx1))


In [7]:
time_array = []
%julia r = []; v=[]; param = [];
for idx0 in idx:
    start = epochs[idx0]
    stop = epochs[idx0 + 1]
    T = np.array((CNOFS_Data.index[(CNOFS_Data.index >= start) & (CNOFS_Data.index < stop)] - start)/dt.timedelta(seconds=1))
    for t in T:
        time_array.append(start + dt.timedelta(seconds= float(t)))
    %julia tle_str = "CNOFS\n" * py"Data[2*idx0]" * py"Data[2*idx0 + 1]";
    %julia tle = SatelliteToolbox.read_tle_from_string(tle_str)
    %julia orbp = SatelliteToolbox.init_orbit_propagator(Val(:sgp4),tle[1])
    %julia T = $T
    %julia for t in T; r0,v0 = SatelliteToolbox.propagate!(orbp,t); push!(r,r0); push!(v,v0);push!(param,[orbp.sgp4d.a_k,orbp.sgp4d.e_k,orbp.sgp4d.i_k,orbp.sgp4d.Ω_k, orbp.sgp4d.ω_k,orbp.sgp4d.M_k,orbp.sgp4d.n_k,orbp.sgp4d.bstar]); end;
r = %julia r
v = %julia v
param = %julia param 
time_array = pd.DatetimeIndex(time_array)

In [8]:
semi_major_axis = [param[i][0] for i in range(len(param))];
ecc = [param[i][1] for i in range(len(param))];
inc = [param[i][2] for i in range(len(param))];
raan = [param[i][3] for i in range(len(param))];
argp = [param[i][4] for i in range(len(param))];
mean_ano = [param[i][5] for i in range(len(param))]; 
mean_motion = [param[i][6] for i in range(len(param))];
bstar = [param[i][7] for i in range(len(param))];
X = [r[i][0] for i in range(len(param))];
Y = [r[i][1] for i in range(len(param))];
Z = [r[i][2] for i in range(len(param))];
Vx = [v[i][0] for i in range(len(param))];
Vy = [v[i][1] for i in range(len(param))];
Vz = [v[i][2] for i in range(len(param))];


In [12]:
lat = [];lon = [];alt = [];
for i in range(10):
    t = Time(time_array[i],format = 'datetime')
    teme_p = (r[i][0],r[i][1],r[i][2])
    teme_v = (v[i][0],v[i][1],v[i][2])
    teme_p = CartesianRepresentation(teme_p * u.m)
    teme_v = CartesianDifferential(teme_v *u.m/u.s)
    teme = TEME(teme_p.with_differentials(teme_v),obstime = t)
    itrs = teme.transform_to(ITRS(obstime = t))
    location = itrs.earth_location
    loc = location.geodetic
    lat.append(loc.lat.value)
    lon.append(loc.lon.value)
    alt.append(loc.height.value)
end = time.time()

In [9]:
df_in = pd.DataFrame({
                    # "lat":lat,
                   # "lon":lon,
                   # "alt":alt,
                   "semi_major_axis":semi_major_axis,
                  "ecc":ecc,
                  "inc":inc,
                  "raan":raan,
                  "argp":argp,
                  "mean_ano":mean_ano,
                  "mean_motion":mean_motion,
                  "bstar":bstar,
                  "X":X,"Y":Y,"Z":Z,
                      "Vx":Vx,"Vy":Vy,"Vz":Vz},index = time_array)
df_in = df_in[(df_in.index >= CNOFS_Data.index[0]) & (df_in.index <= CNOFS_Data.index[-1])]


In [10]:
filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\"
os.chdir(filepath)
filename = "SGP4_prop_reg_"+sat + '_'+str(start.month).zfill(2) + '_' + str(start.year) + ".csv"
df_in.to_csv(filename)